# Results

## Preliminaries

In [2]:
# Library Imports
import os
import csv
import sys
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statistics

In [ ]:
# Funcion Defs
def plot_mapper_times(csvs):
    '''
    
    '''
    times = {}
    for item in csvs:
        tmp = csvs[item]
        tmp = tmp[tmp['function'] ==  'mapper']

        tmp_time = []
        name = f'size_{item.split("_")[0]}'

        times[name] = [[], []]
        for elem in pd.unique(tmp['id']):
            tmp2 = tmp[tmp['id'] == elem]
            mapper_time = float(tmp2['time'].iloc[[-1]].values[0]) - float(tmp2['time'].iloc[[0]].values[0]) 
            tmp_time.append(mapper_time)
            times[name][0].append(tmp2['time'].iloc[[-1]].values[0] - tmp2['time'].iloc[[0]].values[0]) 
            times[name][1].append(tmp2['node'].iloc[[-1]].values[0])
            #times[name][0].sort()

        print(f'Job: {name}. Average Time: {sum(tmp_time)/len(tmp_time)} - Median: {statistics.median(tmp_time)}')


    for k in times:
        bp = go.Figure()
        bp.add_trace(go.Box(y=times[k][0], 
                            name=f'Job {k}', 
                            boxpoints='all', 
                            text=times[k][1]))

        bp.show()
        
def eficiency(csvs):
    '''
    
    '''
    pass

## MINIO Data

In [ ]:
prefix = 'SimulatedData'
csvs = [f'{prefix}/{file_name}' for file_name in os.listdir('SimulatedData/') if file_name.endswith('usage.csv')]
times = {}
print(csvs)
for item in csvs:
    tmp = pd.read_csv(item, delimiter='|')
    tmp = tmp[tmp['function'] ==  'mapper']

    tmp_time = []
    name = f'size_{item.split("/")[1].split("_")[0]}'
    
    times[name] = [[], []]
    for elem in pd.unique(tmp['id']):
        tmp2 = tmp[tmp['id'] == elem]
        mapper_time = float(tmp2['time'].iloc[[-1]].values[0]) - float(tmp2['time'].iloc[[0]].values[0]) 
        tmp_time.append(mapper_time)
        times[name][0].append(tmp2['time'].iloc[[-1]].values[0] - tmp2['time'].iloc[[0]].values[0]) 
        times[name][1].append(tmp2['node'].iloc[[-1]].values[0])
        #times[name][0].sort()

    print(f'Job: {name}. Average Time: {sum(tmp_time)/len(tmp_time)} - Median: {statistics.median(tmp_time)}')

In [ ]:
times

In [ ]:
# Box Plot
#bp = go.Figure()
nodes = ['wn1.localdomain', 'wn2.localdomain', 'wn3.localdomain',
         'wn4.localdomain', 'wn5.localdomain', 'wn6.localdomain']
colors = ['red', 'green', 'blue', 'black', 'orange', 'purple']


for k in times:
    bp = go.Figure()
    bp.add_trace(go.Box(y=times[k][0], 
                        name=f'Job {k}', 
                        boxpoints='all', 
                        text=times[k][1]))
                        #marker=dict(color=colors
                        #            ),
                        #mode='markers'))

    bp.show()

In [ ]:
times['size_8'][1]

In [3]:
df = pd.read_csv(csvs[0] , delimiter='|')

mappers = df[df['function'] == 'mapper']

cpu_figs = make_subplots(rows=2, cols=2, 
                         subplot_titles = ['Mapper 0','Mapper 7','Mapper 8','Mapper 12'],
                         x_title='Time (s)', y_title='CPU Usage (%)')
        
mem_figs = make_subplots(rows=2, cols=2, 
                         subplot_titles = ['Mapper 0','Mapper 1','Mapper 2','Mapper 3'],
                         x_title='Time (s)', y_title='Memory Usage (%)')
row = 1
col = 1


for id in ['0_0', '7_7', '8_8', '12_12']:
    mapper_data = mappers[mappers['id'] == id]
    
    cpu_figs.add_trace(
        go.Scatter(x=mapper_data['time'],
                   y=mapper_data['cpu_percent'],
                   showlegend=False),
        row=row, col=col
    )
    
    mem_figs.add_trace(
        go.Scatter(x=mapper_data['time'],
                   y=mapper_data['mem_percent'],
                   showlegend=False),
        row=row, col=col
    )
    
    col += 1
    if col > 2:
        col = 1
        row +=1
cpu_figs.show()
#mem_figs.show()

NameError: name 'csvs' is not defined

In [ ]:
df2 = pd.read_csv('264285c4-098b-433f-90ae-e6c91eb68716_process.csv' , delimiter='|')
df2 = df2[df2['function'] == 'mapper']
cols = ['function', 'id', 'phase', 'node'] + [col_name for col_name in df2 if col_name.startswith('ctx_') ]
df_ctx = df2[cols]
df_ctx = df_ctx[df_ctx['phase'] == 'end']
df_ctx.sort_values('ctx_involuntary')

## CERN Data

In [17]:
#cpu1_1 = pd.read_csv('', delimiter='')
cpu1_2 = pd.read_csv('cpu_bound_1/28eb5163-faf9-485c-85e7-ed594cf7994f_usage.csv', delimiter='|')
cpu1_4 = pd.read_csv('cpu_bound_1/deea2089-6ee4-4a7b-a8a0-1f25b0083f8c_usage.csv', delimiter='|')
cpu1_8 = pd.read_csv('cpu_bound_1/c3ce453f-9869-4482-9a87-abc7ad1f9bc6_usage.csv', delimiter='|')
cpu1_16= pd.read_csv('cpu_bound_1/21bcd8bb-7d0b-4f32-b998-a71ddad2318b_usage.csv', delimiter='|')

csvs = { 'cpu1-16': cpu1_16, 'cpu1-8': cpu1_8, 'cpu1-4': cpu1_4, 'cpu1-2': cpu1_2}
plot_mapper_times(csvs)

Job: size_cpu1-16. Average Time: 76.5625 - Median: 77.75
Job: size_cpu1-8. Average Time: 143.625 - Median: 144.25
Job: size_cpu1-4. Average Time: 273.0 - Median: 273.75
Job: size_cpu1-2. Average Time: 555.75 - Median: 555.75


In [18]:
cpu0_98_2 = pd.read_csv('cpu_bound_0.98/2_usage.csv', delimiter='|')
cpu0_98_4 = pd.read_csv('cpu_bound_0.98/4_usage.csv', delimiter='|')
cpu0_98_8 = pd.read_csv('cpu_bound_0.98/8_usage.csv', delimiter='|')
cpu0_98_16= pd.read_csv('cpu_bound_0.98/16_usage.csv', delimiter='|')

csvs = { 'cpu0.98-16': cpu0_98_16, 'cpu0.98-8': cpu0_98_8, 'cpu0.98-4': cpu0_98_4, 'cpu0.98-2': cpu0_98_2}
plot_mapper_times(csvs)

Job: size_cpu0.98-16. Average Time: 83.34375 - Median: 79.25
Job: size_cpu0.98-8. Average Time: 162.625 - Median: 149.0
Job: size_cpu0.98-4. Average Time: 314.875 - Median: 291.5
Job: size_cpu0.98-2. Average Time: 579.5 - Median: 579.5


In [14]:

for item in csvs:
    tmp = csvs[item]
    tmp = tmp[tmp['function'] ==  'reducer']

    tmp_time = 0
    name = f'size_{item.split("_")[0]}'
    times[name] = [[], []]
    for elem in pd.unique(tmp['id']):
        tmp2 = tmp[tmp['id'] == elem]
        print(tmp2)
        reducer_time = float(tmp2['time'].iloc[[-1]].values[0]) - float(tmp2['time'].iloc[[0]].values[0]) 
        print('\n\n')
        tmp_time += reducer_time
        times[name][0].append(tmp2['time'].iloc[[-1]].values[0] - tmp2['time'].iloc[[0]].values[0]) 
        times[name][1].append(tmp2['node'].iloc[[-1]].values[0])
        #times[name][0].sort()

    #print('Reduer count: ')    
    #print(f'Job: {name}. Average Time: {sum(tmp_time)/len(tmp_time)} - Median: {statistics.median(tmp_time)}')
    print(tmp_time)

     function   id  time  cpu_percent  mem_percent             node
2466  reducer  0_0   0.0          0.0     0.000062  wn6.localdomain
2467  reducer  0_0   0.5         89.6     0.360636  wn6.localdomain
2468  reducer  0_0   1.0         86.9     0.442859  wn6.localdomain



     function    id  time  cpu_percent  mem_percent             node
2469  reducer  0_15   0.0          0.0     0.000062  wn5.localdomain
2470  reducer  0_15   0.5         87.8     0.386184  wn5.localdomain



     function   id  time  cpu_percent  mem_percent             node
2471  reducer  0_1   0.0          0.0     0.000062  wn5.localdomain
2472  reducer  0_1   0.5         87.8     0.387105  wn5.localdomain
2473  reducer  0_1   1.0         82.8     0.475645  wn5.localdomain



     function   id  time  cpu_percent  mem_percent             node
2474  reducer  0_3   0.0          0.0     0.000062  wn6.localdomain
2475  reducer  0_3   0.5         87.8     0.373945  wn6.localdomain
2476  reducer  0_3   1.0         82.




     function   id  time  cpu_percent  mem_percent             node
2340  reducer  5_5   0.0          0.0     0.000075  wn6.localdomain
2341  reducer  5_5   0.5         89.8     0.382459  wn6.localdomain



     function   id  time  cpu_percent  mem_percent             node
2342  reducer  6_6   0.0          0.0     0.000075  wn6.localdomain
2343  reducer  6_6   0.5         87.8     0.394791  wn6.localdomain



     function   id  time  cpu_percent  mem_percent             node
2344  reducer  6_7   0.0          0.0     0.000075  wn3.localdomain
2345  reducer  6_7   0.5         89.8     0.396452  wn3.localdomain
2346  reducer  6_7   1.0         86.9     0.486131  wn3.localdomain



     function   id  time  cpu_percent  mem_percent             node
2347  reducer  7_7   0.0          0.0     0.000075  wn6.localdomain
2348  reducer  7_7   0.5         87.8     0.370519  wn6.localdomain
2349  reducer  7_7   1.0         86.9     0.464470  wn6.localdomain



14.5
     function   id  time  cp

## MINIO Data

In [ ]:
# Load Data
prefix = 'MinioData'

minio_usage = {}
minio_process = {}

for file_name in os.listdir(f'{prefix}/'):
    key = file_name.split('_')[0]
    if file_name.endswith('usage.csv'):
        tmp = pd.read_csv(f'{prefix}/{file_name}', delimiter='|')
        minio_usage[key] = tmp 
    elif file_name.endswith('process.csv'):
        tmp = pd.read_csv(f'{prefix}/{file_name}', delimiter='|')
        minio_process[key] = tmp

In [ ]:
max_times = []

